<a href="https://colab.research.google.com/github/JasonMullen/A.I.-Group-Project/blob/main/A_I_Group_Project(Using_Reinforcment_Learning_To_Discover_Paths_in_a_2_Agent_3D_Transportation_world).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Experiment One**



*   Define the state space of the environment.

*   Initialize the Q-table with all entries set to zero.



*   Set the discount factor (gamma) and learning rate (alpha) parameters.
*   Run the environment for 500 steps using the PRANDOM policy and update the Q-table after each step.


*   For the next 9500 steps:
a. Run the environment using the PGREEDY policy and update the Q-table after each step.
b. Run the environment using the PEXPLOIT policy and update the Q-table after each step.

*   After the final step, analyze the Q-table and compare the performance of the different policies in terms of utility, reward, and coordination between the agents.







In [ ]:
!pip install gym
!pip install --upgrade gym
!pip install gym --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827649 sha256=77912a7edc13354940e3da61525907c86db9449fc026fe541451d4df3289e5e4
  Stored in directory: /root/.cache/pip/wheels/af/2b/30/5e78b8b9599f2a2286a582b8da80594f654bf0e18d825a4405
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency con

In [ ]:
import numpy as np
import gym

# Create the environment
env = gym.make("Taxi-v3")

# Initialize variables
n_observations = env.observation_space.n
n_actions = env.action_space.n
Q_table = np.zeros((n_observations, n_actions))
n_episodes = 10000
max_iter_episode = 100
exploration_proba = 1
exploration_decreasing_decay = 0.001
min_exploration_proba = 0.01
gamma = 0.99
lr = 0.1
rewards_per_episode = []

# Train the agent
for e in range(n_episodes):
    # Reset the environment for a new episode
    current_state = env.reset()
    done = False
    total_episode_reward = 0

    # Run the episode
    for i in range(max_iter_episode):
        # Choose an action using an epsilon-greedy policy
        if np.random.uniform(0, 1) < exploration_proba:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q_table[current_state, :])

        # Take the chosen action and observe the next state and reward
        next_state, reward, done, _ = env.step(action)

        # Update the Q-table using the Q-learning iteration
        Q_table[current_state, action] = (1 - lr) * Q_table[current_state, action] + lr * (
            reward + gamma * np.max(Q_table[next_state, :])
        )

        # Update the total reward for the episode
        total_episode_reward += reward

        # Update the current state
        current_state = next_state

        # If the episode is done, exit the inner loop
        if done:
            break

    # Update the exploration probability
    exploration_proba = max(min_exploration_proba, np.exp(-exploration_decreasing_decay * e))

    # Append the total reward for the episode to the list
    rewards_per_episode.append(total_episode_reward)

# Print the mean reward per thousand episodes
print("Mean reward per thousand episodes")
for i in range(10):
    print((i + 1) * 1000, ": mean episode reward: ", np.mean(rewards_per_episode[1000 * i : 1000 * (i + 1)]))


Mean reward per thousand episodes
1000 : mean episode reward:  -250.027
2000 : mean episode reward:  -34.37
3000 : mean episode reward:  2.254
4000 : mean episode reward:  5.694
5000 : mean episode reward:  7.219
6000 : mean episode reward:  7.396
7000 : mean episode reward:  7.499
8000 : mean episode reward:  7.632
9000 : mean episode reward:  7.532
10000 : mean episode reward:  7.63


**What do the results mean**

The results show the mean episode reward per thousand episodes during the training of the Q-learning agent to solve the Taxi-v3 environment.

The mean episode reward is a measure of how well the agent is performing. In this case, the reward is negative, which means that the agent is not doing very well at the beginning of the training. As the number of episodes increases, the mean episode reward improves, and the agent learns to perform better.

By the end of the training, the mean episode reward is positive, which indicates that the agent has learned to solve the environment and achieve a high reward on average. However, the final mean episode reward is not very high, which suggests that the agent may not be optimal or that the training could be improved.

Overall, these results provide a quantitative measure of the performance of the Q-learning agent during training and can be used to evaluate and compare different algorithms and hyperparameters.

# **Experiment Two**



*   Initialize the Q-table with zeros for each state-action pair


*   Set the learning rate alpha to 0.3 and the discount rate gamma to 0.5


*   Set the initial policy to PRANDOM and run it for 500 steps


*   Update the policy to PEXPLOIT and run it for 9500 steps

*   Measure the performance of the system in terms of total reward obtained and the number of steps taken to reach the terminal state.



*   Analyze the quality of the coordination between the two agents and how close they get towards the optimal subdivision of work.



*   Analyze the learned paths and their efficiency.



*   Evaluate the performance of SARSA compared to traditional Q-learning by comparing the results to Experiment 1c.



*  Repeat the experiment twice with different random generator seeds and report the results of both runs.


*   Interpret the experimental results, including the final Q-tables and any observed attractive paths.

*   Provide evidence of the running of the system using screenshots.



*   Write a report summarizing the findings of the project and submit the source code of the software along with the report.








Creation of a SARSA algorithm and compare its performance to Q-learning.

In [ ]:
#Jason 4-16-23
import numpy as np
import gym
from gym.envs.registration import register
from gym.envs.toy_text.frozen_lake import FrozenLakeEnv

# Create the custom 3x3 map
custom_map = [
    "SFF",
    "FHF",
    "FFG",
]

env = gym.make("Taxi-v3")

# Initialize variables
n_observations = env.observation_space.n
n_actions = env.action_space.n
Q_table_q_learning = np.zeros((n_observations, n_actions))
Q_table_sarsa = np.zeros((n_observations, n_actions))
n_episodes = 9500
max_iter_episode = 100
exploration_proba = 1
exploration_decreasing_decay = 0.001
min_exploration_proba = 0.01
gamma = 0.99
lr = 0.1
rewards_per_episode_q_learning = []
rewards_per_episode_sarsa = []

def choose_action_epsilon_greedy(state, Q_table, exploration_proba):
    if np.random.uniform(0, 1) < exploration_proba:
        action = env.action_space.sample()
    else:
        action = np.argmax(Q_table[state, :])
    return action

# Train the agent
for e in range(n_episodes):
    # Reset the environment for a new episode
    current_state_q_learning = env.reset()
    current_state_sarsa = current_state_q_learning
    done_q_learning = False
    done_sarsa = False
    total_episode_reward_q_learning = 0
    total_episode_reward_sarsa = 0

    # Choose the initial action for SARSA
    action_sarsa = choose_action_epsilon_greedy(current_state_sarsa, Q_table_sarsa, exploration_proba)

    # Run the episode
    for i in range(max_iter_episode):
        # Q-Learning
        if not done_q_learning:
            # Choose an action using an epsilon-greedy policy
            action_q_learning = choose_action_epsilon_greedy(current_state_q_learning, Q_table_q_learning, exploration_proba)

            # Take the chosen action and observe the next state and reward
            next_state_q_learning, reward_q_learning, done_q_learning, _ = env.step(action_q_learning)

            # Update the Q-table using the Q-learning iteration
            Q_table_q_learning[current_state_q_learning, action_q_learning] = (1 - lr) * Q_table_q_learning[current_state_q_learning, action_q_learning] + lr * (reward_q_learning + gamma * np.max(Q_table_q_learning[next_state_q_learning, :]))

            # Update the total reward for the episode
            total_episode_reward_q_learning += reward_q_learning

            # Update the current state
            current_state_q_learning = next_state_q_learning

        # SARSA
        if not done_sarsa:
            # Take the chosen action and observe the next state and reward
            next_state_sarsa, reward_sarsa, done_sarsa, _ = env.step(action_sarsa)

            # Choose the next action using
            next_action_sarsa = choose_action_epsilon_greedy(next_state_sarsa, Q_table_sarsa, exploration_proba)

            # Update the Q-table using the SARSA iteration
                        # Update the Q-table using the SARSA iteration
            Q_table_sarsa[current_state_sarsa, action_sarsa] = (1 - lr) * Q_table_sarsa[current_state_sarsa, action_sarsa] + lr * (reward_sarsa + gamma * Q_table_sarsa[next_state_sarsa, next_action_sarsa])

            # Update the total reward for the episode
            total_episode_reward_sarsa += reward_sarsa

            # Update the current state and action
            current_state_sarsa = next_state_sarsa
            action_sarsa = next_action_sarsa

        # If both episodes are done, exit the inner loop
        if done_q_learning and done_sarsa:
            break

    # Update the exploration probability
    exploration_proba = max(min_exploration_proba, np.exp(-exploration_decreasing_decay * e))

    # Append the total reward for the episode to the list
    rewards_per_episode_q_learning.append(total_episode_reward_q_learning)
    rewards_per_episode_sarsa.append(total_episode_reward_sarsa)

# Print the mean reward per thousand episodes
print("Mean reward per thousand episodes")
for i in range(9):
    print(f"Q-learning {(i+1)*1000}: mean episode reward: ", np.mean(rewards_per_episode_q_learning[1000*i:1000*(i+1)]))
    print(f"SARSA {(i+1)*1000}: mean episode reward: ", np.mean(rewards_per_episode_sarsa[1000*i:1000*(i+1)]))


Mean reward per thousand episodes
Q-learning 1000: mean episode reward:  -266.231
SARSA 1000: mean episode reward:  -268.398
Q-learning 2000: mean episode reward:  -97.956
SARSA 2000: mean episode reward:  -100.371
Q-learning 3000: mean episode reward:  -3.274
SARSA 3000: mean episode reward:  -5.373
Q-learning 4000: mean episode reward:  4.549
SARSA 4000: mean episode reward:  0.949
Q-learning 5000: mean episode reward:  6.055
SARSA 5000: mean episode reward:  1.953
Q-learning 6000: mean episode reward:  6.197
SARSA 6000: mean episode reward:  1.733
Q-learning 7000: mean episode reward:  6.117
SARSA 7000: mean episode reward:  1.872
Q-learning 8000: mean episode reward:  6.41
SARSA 8000: mean episode reward:  1.746
Q-learning 9000: mean episode reward:  5.746
SARSA 9000: mean episode reward:  1.866


**What do these results mean **

The results show the mean episode reward per thousand episodes during the training of the Q-learning and SARSA agents to solve the Taxi-v3 environment.

Comparing the mean episode rewards between the two agents, we can see that SARSA tends to perform slightly worse than Q-learning throughout the training. However, the difference in performance is not significant, and both agents are able to achieve positive rewards towards the end of the training.

Both agents start with a negative mean episode reward in the first thousand episodes and gradually improve over time. This indicates that the agents are initially exploring and learning about the environment, which leads to negative rewards. As the agents learn more about the environment, they are able to improve their performance and achieve higher rewards.

By the end of the training, both agents are able to achieve mean episode rewards of around 6, which suggests that they have learned to solve the environment and achieve a high reward on average.

Overall, these results provide a quantitative measure of the performance of the Q-learning and SARSA agents during training and can be used to evaluate and compare the performance of these two algorithms.

In [ ]:
print("Q-table for Q-learning:")
print(Q_table_q_learning)

print("Q-table for SARSA:")
print(Q_table_sarsa)

Q-table for Q-learning:
[[  6.21952816  11.98501659   1.41041039  25.56463782 205.55213081
   20.12828214]
 [ -1.60620019 225.14462389  13.96943395  66.32631203  -9.03937091
   -3.3460879 ]
 [ 29.80582314  -0.95680429  -5.86716074  18.50997418 236.42246478
    5.93572816]
 ...
 [ -1.36201528  -1.25394346   7.11269074  -1.46250508  -5.45537692
   -4.86370133]
 [ -2.7390036   14.38842664  -2.7402219   -2.71466454 -10.15592436
   -9.20757539]
 [  7.27397217  29.80378255   2.57524108 334.43242336  70.49300804
   38.11203029]]
Q-table for SARSA:
[[ 17.17333382  21.77528822   7.20819777  25.98165106 108.0644137
   12.86934036]
 [ -5.61368941 -14.7808002  -14.67073161  -9.37846135 195.38888305
   -7.71397571]
 [ -9.85723572  -3.30291886  -4.04421006   8.01947961 104.94341223
  -16.98671189]
 ...
 [ -2.8470787   -2.91890192   5.9937394   -2.89489761  -6.10397092
   -7.39346508]
 [ -5.34067887  -5.16867285  -5.79496059  -5.4048239  -11.89501743
  -14.44537048]
 [ 25.63252478  12.76448277  26.74

**What does the code mean**

These Q-tables show the expected total reward for each possible state-action pair in the Taxi-v3 environment, as learned by the Q-learning and SARSA agents during training.

Looking at the Q-table for Q-learning, we can see that the values in the table have generally increased over time, which indicates that the agent is learning how to navigate the environment more effectively. Specifically, we can see that the values for some of the actions (e.g. picking up or dropping off a passenger) are much higher than the values for other actions (e.g. moving in an empty square), which suggests that the agent is learning to prioritize certain actions in order to maximize its reward.

The Q-table for SARSA shows similar patterns, with increasing values over time and higher values for some actions than others. However, there are some differences between the two tables, which may be due to differences in the learning algorithms used. For example, we can see that the values for some actions in the Q-table for SARSA are higher than the values for the same actions in the Q-table for Q-learning. This could be because SARSA takes into account the next action that it will take in addition to the next state and reward, whereas Q-learning only takes into account the maximum value of the next state-action pair.

Overall, the Q-tables give us a sense of how well the agents have learned to navigate the environment and choose actions that lead to higher rewards. However, they are only one way of evaluating the performance of the agents, and other metrics such as the mean reward per episode may also be useful in assessing their effectiveness.

In Experiment 2, the code compares the performance of Q-learning and SARSA, two reinforcement learning algorithms, in solving a custom 3x3 FrozenLake environment.

The code follows these steps:

A custom 3x3 FrozenLake environment is created with a specific layout (using custom_map). The environment is registered with the gym library, and an environment instance is created using gym.make().

Initialization of required variables, such as the number of observations and actions, Q-tables for both Q-learning and SARSA, and hyperparameters like exploration probability, learning rate, and discount factor.

The agent is trained for a specific number of episodes (n_episodes). Each episode consists of these steps:

a. Reset the environment and initialize the current states for Q-learning and SARSA, as well as the done flags and total rewards for each algorithm.

b. Choose the initial action for SARSA using the epsilon-greedy policy.

c. Run the episode using a loop with a maximum number of iterations (max_iter_episode). For each iteration:

i. Q-Learning:
1. Choose an action using the epsilon-greedy policy.
2. Perform the chosen action, observe the next state and reward.
3. Update the Q-table using the Q-learning update rule.
4. Update the total reward for the episode and the current state.

ii. SARSA:
1. Perform the chosen action, observe the next state and reward.
2. Choose the next action using the epsilon-greedy policy.
3. Update the Q-table using the SARSA update rule.
4. Update the total reward for the episode, the current state, and the current action.

iii. If both episodes (Q-learning and SARSA) are done, break out of the loop.

d. Update the exploration probability for the epsilon-greedy policy using an exponential decay.

e. Append the total reward for the episode to the respective lists for both Q-learning and SARSA.

After training, the mean reward per thousand episodes is calculated and printed for both Q-learning and SARSA.

The purpose of this experiment is to analyze and compare the performance of Q-learning and SARSA algorithms in the custom 3x3 FrozenLake environment. By printing the mean reward per thousand episodes, we can assess how well each algorithm performs and converges to a solution.

# **Experiment Three**




*  In Experiment 3, you will rerun either Experiment 1.c or 2 but with learning rates α=0.1 and α=0.5. The purpose of this experiment is to analyze the effects of using different learning rates on the system's performance.


*   Set the learning rate to α=0.1 and run the chosen experiment from either Experiment 1.c or 2 for 10000 steps.



*   Set the learning rate to α=0.5 and run the chosen experiment from either Experiment 1.c or 2 for 10000 steps.


*   Record the performance metrics such as total reward, average reward per episode, and success rate for both experiments with different learning rates.


*   Analyze the effects of using different learning rates on the system's performance, focusing on how the learning rate affects the convergence rate and the quality of the learned policies.


*   Identify any trade-offs between using a high or low learning rate.

*   Summarize and interpret the results obtained from both experiments.




In [ ]:
#Jason 4-16-23
import numpy as np
import gym

# Create the environment
env = gym.make("Taxi-v3")

# Initialize variables
n_observations = env.observation_space.n
n_actions = env.action_space.n
n_episodes = 10000
max_iter_episode = 100
exploration_proba = 1
exploration_decreasing_decay = 0.001
min_exploration_proba = 0.01
gamma = 0.99
learning_rates = [0.1, 0.5]

def choose_action_epsilon_greedy(state, Q_table, exploration_proba):
    if np.random.uniform(0, 1) < exploration_proba:
        action = env.action_space.sample()
    else:
        action = np.argmax(Q_table[state, :])
    return action

for lr in learning_rates:
    print(f"Experiment with learning rate α={lr}")
    Q_table_q_learning = np.zeros((n_observations, n_actions))
    rewards_per_episode_q_learning = []
    success_count = 0

    # Train the agent
    for e in range(n_episodes):
        # Reset the environment for a new episode
        current_state_q_learning = env.reset()
        done_q_learning = False
        total_episode_reward_q_learning = 0

        # Run the episode
        for i in range(max_iter_episode):
            # Q-Learning
            if not done_q_learning:
                # Choose an action using an epsilon-greedy policy
                action_q_learning = choose_action_epsilon_greedy(current_state_q_learning, Q_table_q_learning, exploration_proba)

                # Take the chosen action and observe the next state and reward
                next_state_q_learning, reward_q_learning, done_q_learning, _ = env.step(action_q_learning)

                # Update the Q-table using the Q-learning iteration
                Q_table_q_learning[current_state_q_learning, action_q_learning] = (1 - lr) * Q_table_q_learning[current_state_q_learning, action_q_learning] + lr * (reward_q_learning + gamma * np.max(Q_table_q_learning[next_state_q_learning, :]))

                # Update the total reward for the episode
                total_episode_reward_q_learning += reward_q_learning

                # Update the current state
                current_state_q_learning = next_state_q_learning

            if done_q_learning:
                break

        # Update the exploration probability
        exploration_proba = max(min_exploration_proba, np.exp(-exploration_decreasing_decay * e))

        # Append the total reward for the episode to the list
        rewards_per_episode_q_learning.append(total_episode_reward_q_learning)

        if reward_q_learning >= 20:
            success_count += 1

    print(f"Total reward: {np.sum(rewards_per_episode_q_learning)}")
    print(f"Average reward per episode: {np.mean(rewards_per_episode_q_learning)}")
    print(f"Success rate: {success_count / n_episodes * 100}%")
    print("\n")


Experiment with learning rate α=0.1
Total reward: -236018
Average reward per episode: -23.6018
Success rate: 92.16%


Experiment with learning rate α=0.5
Total reward: -117738
Average reward per episode: -11.7738
Success rate: 96.74000000000001%




# **Experiment Four**



*   Somewhat similar to Experiment 1c or 2; except use =0.3 and =0.5 in conjunction with either Q-earning or SARSA.
*   Run PRANDOM for the first 500 steps; next, you run PEXPLOIT; however, after a terminal state is reached the third time change the 2 pickup locations to: (2, 3, 3) and (1, 3, 1); the drop off locations and the Q-table remain unchanged;
*   Finally continue running PEXPLOIT with the new pickup locations until the agent reaches a terminal state the 6th time.
*   When interpreting the results of this experiment center on analyzing on how well the learning strategy was able to adapt to the change of the pickup locations and to which extend it was able to learn new paths and unlearn old paths





In [ ]:
import gym
import numpy as np

def choose_action(state, q_table, epsilon):
    if np.random.uniform(0, 1) < epsilon:
        action = np.random.choice(env.action_space.n)
    else:
        action = np.argmax(q_table[state])
    return action

def learn(state, action, reward, next_state, next_action, q_table, alpha, gamma):
    predict = q_table[state, action]
    target = reward + gamma * q_table[next_state, next_action]
    q_table[state, action] += alpha * (target - predict)

def run_experiment(env, epsilon, alpha, gamma, num_random_steps, num_exploit_steps):
    q_table = np.zeros((env.observation_space.n, env.action_space.n))

    # Run PRANDOM policy for num_random_steps
    state = env.reset()
    for _ in range(num_random_steps):
        action = env.action_space.sample()
        next_state, reward, done, _ = env.step(action)
        learn(state, action, reward, next_state, choose_action(next_state, q_table, epsilon), q_table, alpha, gamma)
        state = next_state if not done else env.reset()

    # Run PEXPLOIT policy until terminal state is reached 3 times
    terminal_count = 0
    state = env.reset()
    while terminal_count < 3:
        action = choose_action(state, q_table, epsilon)
        next_state, reward, done, _ = env.step(action)
        learn(state, action, reward, next_state, choose_action(next_state, q_table, epsilon), q_table, alpha, gamma)
        state = next_state if not done else env.reset()
        terminal_count += done

    # Change pickup locations
    env.change_pickup_locations([(2, 3, 3), (1, 3, 1)])

    # Run PEXPLOIT policy until terminal state is reached 6 times
    terminal_count = 0
    state = env.reset()
    while terminal_count < 6:
        action = choose_action(state, q_table, epsilon)
        next_state, reward, done, _ = env.step(action)
        learn(state, action, reward, next_state, choose_action(next_state, q_table, epsilon), q_table, alpha, gamma)
        state = next_state if not done else env.reset()
        terminal_count += done

    return q_table

# Initialize the environment
env = gym.make('Taxi-v3')

# Set the experiment parameters
epsilon = 0.3
alpha = 0.5
gamma = 0.99
num_random_steps = 500
num_exploit_steps = 10000

# Run the experiment
q_table = run_experiment(env, epsilon, alpha, gamma, num_random_steps, num_exploit_steps)

# Close the environment
env.close()


/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


AttributeError: ignored

In [ ]:
import numpy as np
import gym
from collections import defaultdict

# Create the environment
env = gym.make("Taxi-v3")

# Initialize variables
n_observations = env.observation_space.n
n_actions = env.action_space.n
n_episodes = 10000
max_iter_episode = 100
exploration_proba = 1
exploration_decreasing_decay = 0.001
min_exploration_proba = 0.01
gamma = 0.99
learning_rates = [0.3, 0.5]

# Modify the environment
def modify_taxi_environment(env, new_pickup_locations):
    env.locs = new_pickup_locations

# Initialize new pickup locations
new_pickup_locations = [(2, 3, 3), (1, 3, 1)]

def choose_action_epsilon_greedy(state, Q_table, exploration_proba):
    if np.random.uniform(0, 1) < exploration_proba:
        action = env.action_space.sample()
    else:
        action = np.argmax(Q_table[state, :])
    return action

for lr in learning_rates:
    print(f"Experiment with learning rate α={lr}")
    Q_table_q_learning = np.zeros((n_observations, n_actions))
    rewards_per_episode_q_learning = []
    success_count = 0
    terminal_state_count = 0
    steps = 0

    # Train the agent
    for e in range(n_episodes):
        if terminal_state_count == 6:
            break

        # Reset the environment for a new episode
        current_state_q_learning = env.reset()
        done_q_learning = False
        total_episode_reward_q_learning = 0

        # Run the episode
        for i in range(max_iter_episode):
            if terminal_state_count == 6:
                break

            # Q-Learning
            if not done_q_learning:
                if steps < 500:
                    action_q_learning = env.action_space.sample()
                else:
                    action_q_learning = choose_action_epsilon_greedy(current_state_q_learning, Q_table_q_learning, exploration_proba)

                # Take the chosen action and observe the next state and reward
                next_state_q_learning, reward_q_learning, done_q_learning, _ = env.step(action_q_learning)

                # Update the Q-table using the Q-learning iteration
                Q_table_q_learning[current_state_q_learning, action_q_learning] = (1 - lr) * Q_table_q_learning[current_state_q_learning, action_q_learning] + lr * (reward_q_learning + gamma * np.max(Q_table_q_learning[next_state_q_learning, :]))

                # Update the total reward for the episode
                total_episode_reward_q_learning += reward_q_learning

                # Update the current state
                current_state_q_learning = next_state_q_learning

            if done_q_learning:
                terminal_state_count += 1

                if terminal_state_count == 3:
                    modify_taxi_environment(env, new_pickup_locations)

            steps += 1

    print(f"Total reward: {np.sum(rewards_per_episode_q_learning)}")
    print(f"Average reward per episode: {np.mean(rewards_per_episode_q_learning)}")
    print(f"Terminal states reached: {terminal_state_count}")
    print("\n")


Experiment with learning rate α=0.3
Total reward: 0.0
Average reward per episode: nan
Terminal states reached: 6


Experiment with learning rate α=0.5


/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Total reward: 0.0
Average reward per episode: nan
Terminal states reached: 6


